In [1]:
import sys
sys.path.insert(0, '/home/vipul/llm/iterate')
print(sys.path)

['/home/vipul/llm/iterate', '/home/vipul/miniconda3/envs/llm/lib/python310.zip', '/home/vipul/miniconda3/envs/llm/lib/python3.10', '/home/vipul/miniconda3/envs/llm/lib/python3.10/lib-dynload', '', '/home/vipul/miniconda3/envs/llm/lib/python3.10/site-packages']


In [2]:
from dataset import JSONLDataset, TabularDataset, PickleDataset
import models.openai as openai
from util import parse_example, parse_tsv_example, parse_qaner_example, score_sets
import numpy as np
import time
import re

from dotenv import load_dotenv

import argparse
import random
from tqdm.auto import tqdm
import os, pdb
import json
import shutil
import logging
from datetime import datetime
import signal

/home/vipul/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def run_prompt(prompt, model):
    completion = model.complete(prompt)
    if completion is None or completion == '':
        return "Could not complete prompt"
    return completion

def construct_prompt(srcfile, promptfile, egs):
    ds = JSONLDataset(srcfile)
    egs = []
    for idx in eg_idxs:
        egs.append(ds[idx])
    
    for eg in egs:
        for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
            if ptok == gtok:
                print(ptok, end=' ')
            else:
                print(f'[{ptok}]', end=' ')
        print()

def get_prompt_eg_acc(srcfile, eg_idxs):
    ds = JSONLDataset(srcfile)
    egs = []
    for idx in eg_idxs:
        egs.append(ds[idx])
    
    for eg in egs:
        for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
            if ptok == gtok:
                print(ptok, end=' ')
            else:
                print(f'[{ptok}->{gtok}]', end=' ')
        print()
    
def create_model():
    load_dotenv('../../.env')
    openai.setup_api_key(os.environ.get('OPENAI_API_KEY'))

    model_args = openai.ChatGPT.DEFAULT_ARGS
    model_args['engine'] = 'gpt-4-turbo'
    model_args['request_timeout'] = 200
    model_args['temperature'] = 0
    return openai.ChatGPT(model_args)

def parse_response(response):
    lines = response.strip().split('\n')
    pred = [a.split('\t') for a in lines if '\t' in a]
    return list(zip(*pred))

In [13]:
eg_1_prompt = """You are an NLP assistant whose purpose is to solve Natural Language Inference (NLI) problems. NLI is the task of determining the inference relation between two (short, ordered) texts: entailment, contradiction, or neutral. Answer as concisely as possible in the same format as the examples below:

Premise: Ah, huk chaypi allinqa apakurqa allin qawasqayqa paniypa ñawpaq yuyariyninmi, chaypas hina hipa pampapim karqa. , Hypothesis: Yuyaruniqa hipa pampapi huk ima apakusqantam. , Answer: entailment

Premise: Yaykuykuptiykuqa punkukunaqa wichqasqam kachkarqa. , Hypothesis: Punku wichqasqa kachkaptinpas yaykurqanikum. , Answer: entailment

Premise: Yanapawaqniy atiq sispasmi hatun llaqtapa waklawninpiraq tiyan. , Hypothesis: Yanapawaqniy warmi warman 5 millas nisqan karupiraq tiyan. , Answer: entailment

Premise: Manam mayman risqanta yacharqanikuchu. , Hypothesis: Mayman risqantam yacharqaniku. , Answer: entailment

Premise: Chayna kaptinqa hamutachkanim huktapiwan Ramonawan rimariyta. , Hypothesis: Ramonawanmi huktapiwan rimarqani. , Answer: entailment

Premise: Ripukusqañam hinaspam amaña llakikunaypaq niwarqa. , Hypothesis: Ama llakikunaytam niwarqa. , Answer: entailment

Premise: Ichapasyá huk kaq mana yachasqaymanta hamun ichaqa , Hypothesis: Apurawtam hamun, ichaqa maymanta hamusqanta yachanim. , Answer: entailment

Premise: Locust Hill oh awriki, ari, kusa , Hypothesis: Locust Hill nisqaqa allinmi. , Answer: contradiction

Premise: Oh, payllam isqun iskay iskayraq regulador nisqapi inyecciónta qinaq karqa. , Hypothesis: Martes punchawtam inyector nisqata hinarqani. , Answer: """

input: I cha der ihri Telefonnummere gä , de nimmsch mou unverbindlech Kontakt uuf .

Examples:

59  
I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .  
PRON AUX [DET->ADP] PROPN VERB PUNCT PRON AUX PRON ADV ADJ CCONJ PRON [VERB->AUX] ADV [ADV->PART] VERB PUNCT   

4  
De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .  
DET NOUN AUX DET NOUN VERB PUNCT DET NOUN [VERB->AUX] DET NOUN VERB AUX PUNCT   

7  
Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .  
DET NOUN AUX ADV AUX PUNCT SCONJ PRON ADV ADV ADJ [ADP->AUX] AUX PUNCT SCONJ PRON ADV AUX AUX PART VERB PUNCT   

70  
Isch das e Sach gsi , bis mer se gfunge hei gha .  
AUX [DET->PRON] DET NOUN [VERB->AUX] PUNCT SCONJ PRON PRON VERB AUX [VERB->AUX] PUNCT   

61  
De Dialäkt muess zu de Gschecht und zum Inhaut vonere Werbig passe .  
DET NOUN AUX ADP DET NOUN CCONJ ADP NOUN ADP NOUN VERB PUNCT   

56  
Mit der Zit hani mi mit mir säuber uf ei Schriibwiis pro Wort aafo einige .  
ADP DET NOUN [PRON->AUX] PRON ADP PRON ADV ADP DET NOUN ADP NOUN VERB VERB PUNCT   

30  
Mit all denä Wörter hani natürli nüt chönä aafangä .  
ADP DET [PRON->DET] NOUN [VERB->AUX] ADV PRON [VERB->AUX] VERB PUNCT   

15  
Aso bini rächt uufgschmissä gsi und dem entschprächend fascht verzwiiflät .  
ADV AUX ADV [VERB->ADJ] AUX CCONJ PRON ADJ ADV VERB PUNCT   



In [17]:
eg_1_idxs = [66, 58, 93, 45, 15, 68, 26, 70]
eg_2_prompt = """You are an NLP assistant whose purpose is to solve Natural Language Inference (NLI) problems. NLI is the task of determining the inference relation between two (short, ordered) texts: entailment, contradiction, or neutral. Answer as concisely as possible in the same format as the examples below:

Premise: Ah, huk chaypi allinqa apakurqa allin qawasqayqa paniypa ñawpaq yuyariyninmi, chaypas hina hipa pampapim karqa. , Hypothesis: Yuyaruniqa hipa pampapi huk ima apakusqantam. , Answer: entailment

Premise: Yaykuykuptiykuqa punkukunaqa wichqasqam kachkarqa. , Hypothesis: Punku wichqasqa kachkaptinpas yaykurqanikum. , Answer: entailment

Premise: Manam mayman risqanta yacharqanikuchu. , Hypothesis: Mayman risqantam yacharqaniku. , Answer: entailment

Premise: Chayna kaptinqa hamutachkanim huktapiwan Ramonawan rimariyta. , Hypothesis: Ramonawanmi huktapiwan rimarqani. , Answer: entailment

Premise: Manam pachay karqachu ima kaqpas ruranaypaq. , Hypothesis: Mana pacha llapan qinanaypaq haypawarqachu , Answer: entailment

Premise: Ripukusqañam hinaspam amaña llakikunaypaq niwarqa. , Hypothesis: Ama llakikunaytam niwarqa. , Answer: entailment

Premise: Ichapasyá huk kaq mana yachasqaymanta hamun ichaqa , Hypothesis: Apurawtam hamun, ichaqa maymanta hamusqanta yachanim. , Answer: entailment

Premise: Locust Hill oh awriki, ari, kusa , Hypothesis: Locust Hill nisqaqa allinmi. , Answer: contradiction

Premise: Oh, payllam isqun iskay iskayraq regulador nisqapi inyecciónta qinaq karqa. , Hypothesis: Martes punchawtam inyector nisqata hinarqani. , Answer: """

In [18]:
model = create_model()

In [19]:
def print_results(prompt, model):
    print(run_prompt(prompt, model))

In [20]:
print_results(eg_2_prompt, model)

neutral


In [12]:
eg_1_prompt

'You are an NLP assistant whose purpose is to solve Natural Language Inference (NLI) problems. NLI is the task of determining the inference relation between two (short, ordered) texts: entailment, contradiction, or neutral. Answer as concisely as possible in the same format as the examples below:\n\nPremise: Ah, huk chaypi allinqa apakurqa allin qawasqayqa paniypa ñawpaq yuyariyninmi, chaypas hina hipa pampapim karqa. , Hypothesis: Yuyaruniqa hipa pampapi huk ima apakusqantam. , Answer: entailment\n\nPremise: Yaykuykuptiykuqa punkukunaqa wichqasqam kachkarqa. , Hypothesis: Punku wichqasqa kachkaptinpas yaykurqanikum. , Answer: entailment\n\nPremise: Yanapawaqniy atiq sispasmi hatun llaqtapa waklawninpiraq tiyan. , Hypothesis: Yanapawaqniy warmi warman 5 millas nisqan karupiraq tiyan. , Answer: contradiction\n\nPremise: Manam mayman risqanta yacharqanikuchu. , Hypothesis: Mayman risqantam yacharqaniku. , Answer: entailment\n\nPremise: Chayna kaptinqa hamutachkanim huktapiwan Ramonawan r